# **Import the relevant libraries :**

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# **Data :**

In [2]:
mnist_dataset,mnist_info= tfds.load(name = 'mnist',with_info = True, as_supervised = True)

2022-04-08 15:56:25.325299: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


2022-04-08 15:56:29.490111: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples

num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

In [4]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs,validation_targets = next(iter(validation_data))

2022-04-08 15:56:29.822842: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-08 15:56:31.106703: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# **Model :**

## **Outline the model :**

In [5]:
input_size = 784
output_size = 10
hidden_layer = 100

model = tf.keras.Sequential ([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer,activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer,activation = 'relu'),
    tf.keras.layers.Dense(output_size,activation = 'softmax')
    ])

## **Choose the optimizer and loss function :**

In [6]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# **Training :**

In [7]:
NUM_EPOCHS = 5

model.fit(train_data,epochs = NUM_EPOCHS,validation_data = (validation_inputs,validation_targets), verbose = 2)

Epoch 1/5
540/540 - 6s - loss: 0.3290 - accuracy: 0.9058 - val_loss: 0.1572 - val_accuracy: 0.9530
Epoch 2/5
540/540 - 4s - loss: 0.1370 - accuracy: 0.9602 - val_loss: 0.1203 - val_accuracy: 0.9655
Epoch 3/5
540/540 - 4s - loss: 0.0981 - accuracy: 0.9706 - val_loss: 0.0885 - val_accuracy: 0.9732
Epoch 4/5
540/540 - 4s - loss: 0.0771 - accuracy: 0.9765 - val_loss: 0.0696 - val_accuracy: 0.9800
Epoch 5/5
540/540 - 4s - loss: 0.0585 - accuracy: 0.9817 - val_loss: 0.0562 - val_accuracy: 0.9842


# **Test the model :**

In [8]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 726ms/step - loss: 0.0799 - accuracy: 0.9766


In [9]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.66%
